<h1>Tasks for text preparation<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [12]:
from math import log, sqrt
import sympy.parsing.sympy_parser
import sympy
import sys
import numpy as np

Пусть в некотором языке есть N=3 слова - А, Б и В. Их ранги - 1, 2 и 3 (нумерация рангов начинается с 1). 

Найдите вероятности встретить каждое из этих слов в тексте при условии, что относительные частоты слов распределены по Ципфу с s = 2.

In [1]:
def word_probability(rank, s, n):
    sum = 0
    for i in range(1, n+1):
        sum = sum + i**(-s)
    q = 1 / (rank**s * sum)
    return q


print(round(word_probability(1, 2, 3), 4))
print(round(word_probability(2, 2, 3), 4))
print(round(word_probability(3, 2, 3), 4))

0.7347
0.1837
0.0816


Мы предлагаем вам проанализировать полученную формулу на следующих предсказаниях модели:


[0.99,0.01] - первый объект классифицируется уверенно правильно, а второй уверенно неправильно

[0.5,0.5] - модель не может принять решения, абсолютно неуверенное предсказание

[0.99,0.45] - первый объект классифицируется уверенно правильно, а второй неуверенно неправильно

[0.65,0.65] - оба объекта классифицируются правильно, но классификатор не очень уверен в принятом решении

Фраза "Модели выгоднее предсказывать А, чем Б" означает, что суммарное значение функции потерь будет ниже, если модель предскажет набор ответов А, по сравнению с ситуацией, в которой она для тех же объектов предскажет набор ответов Б.
 

In [4]:
def BCE(y1, y2):
    return -log(y1) - log(y2)


a = [0.99, 0.01]
b = [0.5, 0.5]
c = [0.99, 0.45]
d = [0.65, 0.65]

print(f'BCE_099_001: {BCE(a[0], a[1])}')
print(f'BCE_05_05: {BCE(b[0], b[1])}')
print(f'BCE_099_045: {BCE(c[0], c[1])}')
print(f'BCE_065_065: {BCE(d[0], d[1])}')

BCE_099_001: 4.615220521841592
BCE_05_05: 1.3862943611198906
BCE_099_045: 0.8085580320712731
BCE_065_065: 0.8615658321849085


Найдите в общем виде производную функции потерь по w BCE и запишите в ответ её формулу. 
Для обозначений используйте латинские буквы y, x, w, b в нижнем регистре.

In [7]:
x, y, w, b = sympy.symbols('x y w b')
res = sympy.diff(-y * sympy.log(1 / (1 + sympy.exp(-w * x - b)))
                 - (1  - y) * sympy.log(1 - 1 / (1 + sympy.exp(-w * x - b))), b)


print(str(sympy.simplify(res)))

(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)


In [8]:
x, y, w, b, c = sympy.symbols('x y w b c')
diff1 = sympy.diff('-y*log(1/(1+exp(-w*x-b)))-(1-y)*log(1-1/(1+exp(-w*x-b)))', w)
diff2 = sympy.diff('c*(w**2+b**2)', w)
simpl1 = sympy.simplify(diff1)
simpl2 = sympy.simplify(diff2)
sample_expr = sympy.parsing.sympy_parser.parse_expr(str(simpl1) + '+' + str(simpl2))
sample_value = sample_expr.evalf(subs=dict(x=0.5, y=1, w=4, b=1, c=1))


print(sample_value)
print(str(diff1) + '+' + str(diff2))
print(str(simpl1) + '+' + str(simpl2))
simple3 = sympy.simplify(diff1+diff2)
print(simple3)

7.97628706341122
-x*y*exp(-b - w*x)/(exp(-b - w*x) + 1) + x*(1 - y)*exp(-b - w*x)/((1 - 1/(exp(-b - w*x) + 1))*(exp(-b - w*x) + 1)**2)+2*c*w
x*(-y*exp(b + w*x) - y + exp(b + w*x))/(exp(b + w*x) + 1)+2*c*w
(2*c*w*exp(b + w*x) + 2*c*w - x*y*exp(b + w*x) - x*y + x*exp(b + w*x))/(exp(b + w*x) + 1)


Напишите функцию для вычисления точечной взаимной информации двух случайных событий.

На вход функция получает два массива из 0 и 1 одинаковой длины - реализации случайных событий. 1 - событие произошло, 0 - не произошло.

В результате функция должна вернуть вещественное число - точечную взаимную информацию событий.

При расчёте следует использовать натуральный логарифм (numpy.log).

In [10]:
def parse_array(s):
    return np.array([int(s.strip()) for s in s.strip().split(' ')])

def read_array():
    return parse_array(sys.stdin.readline())

def calculate_pmi(a, b):
    P_a = np.count_nonzero(a == 1) / a.shape[0]
    P_b = np.count_nonzero(b == 1) / b.shape[0]
    
    c = [i for  i, j in zip(a,b) if i == j == 1]       
    P_a_b = len(c) / a.shape[0]

    # print(c)
    # print(type(c))
    
    return np.log(P_a_b / (P_a * P_b))


a = np.array([1, 0, 0, 1, 1, 0])
b = np.array([1, 0, 0, 0, 1, 0])
pmi_value = calculate_pmi(a, b)


print('{:.6f}'.format(pmi_value))

0.693147


Найдите количество слов, которые встречаются менее, чем в 10 из 10000 документов, если предполагать, что вероятность встретить слово в документе распределена по Ципфу с параметром s=2, количество слов в словаре N=1000. Ранги нумеруются с 1.

In [13]:
z = sum([1/x**2 for x in range(1,1000)])
k = sqrt(1000/z)


print(str(1000-int(k)))

976
